<a href="https://colab.research.google.com/github/annie-lyu/TrainCNN4SignatureVerification/blob/master/Copy_of_ldaseq_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from gensim import corpora
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
import numpy
from gensim.matutils import hellinger
import nltk


In [2]:
import glob
from google.colab import drive
drive.mount('/content/drive')
##folder lists
ls = !ls "/content/drive/My Drive/words/sample"
ls = str(ls).strip("'[")
ls = ls.strip("]'")
entries = ls.split("\\t")
entries

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


['month1', 'month2', 'month3']

In [4]:
## preparation for text preprocessing
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(set(stopwords.words('english'))) + ["us", "also", "said", "would", "could", "mr", "get"]
from nltk.stem import PorterStemmer
ps = PorterStemmer()
stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['my',
 'his',
 'again',
 'yourself',
 'doing',
 'myself',
 'why',
 'we',
 'just',
 'hasn',
 'theirs',
 'shan',
 'before',
 "wasn't",
 'be',
 'both',
 've',
 'no',
 'me',
 'then',
 'aren',
 'has',
 'up',
 'couldn',
 "you'd",
 'not',
 'ourselves',
 'to',
 "mustn't",
 "doesn't",
 'wasn',
 'some',
 'through',
 "couldn't",
 'himself',
 'it',
 'of',
 'ain',
 'between',
 'more',
 'm',
 'because',
 'how',
 'what',
 'by',
 'haven',
 'them',
 "hadn't",
 "she's",
 'she',
 'herself',
 'its',
 "shan't",
 'once',
 'too',
 'themselves',
 'here',
 'i',
 'won',
 "didn't",
 "you're",
 'your',
 "wouldn't",
 'were',
 'if',
 'or',
 'under',
 "haven't",
 'o',
 'had',
 'does',
 'above',
 'mightn',
 'will',
 "shouldn't",
 'doesn',
 's',
 'about',
 'further',
 'shouldn',
 'while',
 "that'll",
 'such',
 "should've",
 'off',
 'weren',
 'only',
 "needn't",
 're',
 'each',
 'below',
 "it's",
 'own',
 'didn',
 'which',
 'wouldn',
 'our',
 'was',
 'have',
 'll',
 'should',
 'few',
 'those',
 'against',
 'and',
 'at

In [5]:
import re
def removePunctuation(text):
    punctuation = '!,;:?"\'.-'
    text = re.sub(r'[{}]+'.format(punctuation),'',text)
    return text.strip().lower()
 

In [6]:
#loading corpus and preprocessing

corpus_list= []
for entry in entries:
    filepath = "/".join(["/content/drive/My Drive/words/sample", entry, "*.txt"])
    data_files = glob.glob(filepath)
    for data_file in data_files:
      document = open(data_file,'r').read().lower()
      text = [word for word in document.split()] 
      corpus = [removePunctuation(word) for word in text]
      corpus = [word for word in corpus if not word in stop_words]
      corpus_list += [corpus]
len(corpus_list)





1323

In [7]:
#3 Merge corpus into one document
with open("/content/drive/My Drive/words/sample/BleiCorpus.txt", 'w') as File:
  for corpus in corpus_list:
    File.write(" ".join(corpus)+ "\n" )

In [8]:
## establish dictionary 
dictionary = corpora.Dictionary(corpus_list)
dictionary.save("/content/drive/My Drive/words/sample/word_dictionary")


## create vocabulary file
class MyCorpus(object):
    def __iter__(self):
        for line in open("/content/drive/My Drive/words/sample/BleiCorpus.txt"):
            yield dictionary.doc2bow(line.lower().split())

corpus_memory_friendly = MyCorpus()
corpus = [vector for vector in corpus_memory_friendly]
corpora.BleiCorpus.serialize('/content/drive/My Drive/words/sample/news_corpus', corpus)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
## loading and setting
try:
  dictionary = Dictionary.load("/content/drive/My Drive/words/sample/word_dictionary")
except FileNotFoundError as e:
  raise ValueError("SKIP: Please download the Corpus/news_dictionary dataset.")

corpus = bleicorpus.BleiCorpus("/content/drive/My Drive/words/sample/news_corpus")

time_slice = [438, 429, 456]


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=dictionary, time_slice=time_slice, num_topics=5)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/models/ldaseqmodel.py:290: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function i

In [ ]:
#Printing Topics
ldaseq.print_topics(time=0)

In [ ]:
#Topic Evolution
ldaseq.print_topic_times(topic=0)

In [ ]:
## word distribution in a specific document
words = [dictionary[word_id] for word_id, count in corpus[558]]
print (words)

In [ ]:
## A specific document's latent topic distribution
doc = ldaseq.doc_topics(558)
print(doc)

In [ ]:
doc_football_1 = ['economy', 'bank', 'mobile', 'phone', 'markets', 'buy', 'football', 'united', 'giggs'] #make up a news about how football affect economy
doc_football_1 = dictionary.doc2bow(doc_football_1)
doc_football_1 = ldaseq[doc_football_1]
print (doc_football_1)

In [ ]:
## topic distance between documents (hellinger distance: measure similarity of two distribution)
doc_football_2 = ['arsenal', 'fourth', 'wenger', 'oil', 'middle', 'east', 'sanction', 'fluctuation']
doc_football_2 = dictionary.doc2bow(doc_football_2)
doc_football_2 = ldaseq[doc_football_2]
hellinger(doc_football_1, doc_football_2)

In [ ]:
## changing rate of topic factors(words in topics)
ldaseq.print_topic_times(1)

##Note: the chain_variance parameter could affect the result